# COMP7095 - Big Data Management

## Lab 3 MongoDB Part 2: MongoDB NoSQL + Python

### Launching Jupyter Notebook
We are going to switch the remaining parts to Jupyter Notebook. Let's try it with the following steps:

1. Download the `ipynb version` of this lab.
2. save it to the `comp7095` folder on the desktop.
3. Launch the terminal (macOS) or command prompt (Windows).
4. Run the followings commands:

    macOS: \
    <code>cd Desktop/comp7095
    source ./venv/bin/activate
    jupyter notebook
    </code>

    Windows: \
    <code>cd comp7095
    .\venv\Scripts\activate
    jupyter notebook
    </code>
    

5. On the Jupyter home page, select the `ipnb version` of this lab to run it. Then, you will see this page in notebook format.

## Starting MongoDB
Next, we need to start the MongoDB.

macOS:
1. Launch a new terminal and run the following command: \
`brew services start mongodb-community@6.0`


2. Wait for the respond to verify MongoDB is running.
3. Leave the terminal aside.

Windows:
1. Launch a new Command Prompt and run the following command: \
`"C:\Program Files\MongoDB\Server\6.0\bin\mongod.exe" --dbpath="c:\data\db"`


2. Wait for the respond to verify MongoDB is running.
3. Leave the Command Prompt aside.

## Connecting MongoDB
MongoDB is now running locally. We are going to use Python code to connect it.

Import the required packages:
- `MongoClient` is used to establish connect to MongoDB. With the client, we can insert, update, and delete the documents in MongoDB.
- `pprint` provides a capability to “pretty-print” arbitrary Python data structures in a form which can be used as input to the interpreter.

First, we need to import the required packages.

In [ ]:
from pymongo import MongoClient
from pprint import pprint

To connect MongoDB, we need to use the `MongoClient` function with the connection string. The connection string - `mongodb://localhost:27017`, describes the protocol, host, and port. If you want to connect MongoDB running on another host, you should replace `localhost` with the IP address of the host.

In [ ]:
client = MongoClient('mongodb://localhost:27017/')

You can check the existing databases by using the `list_database_names` function.

In [ ]:
client.list_database_names()

To switch database, we only need to provide the database name as follows:

In [ ]:
db = client['mydb']

Let's use the `list_collection_names` function to see what collections the database has.

In [ ]:
db.list_collection_names()

Now, we are going to use the `products` collection.

In [ ]:
products = db['products']

In the last lab, we use `mongosh` to create some documents in the `products` collection. Let's see whether we can get them back in Python.

We call the `find` function, just like what we have done in `mongosh`. The output of the `find` function is in Cursor format. For convenience, we convert the output of the `find` function to a list.

In [ ]:
data = list(products.find())
pprint(data)
    

The following two lines of code is used to delete some documents created by this notebook, just in case you ran this notebook previously.

In [ ]:
products.delete_one({'_id':12})
products.delete_many({'type':'unknown'})

With the argument, the `find` function filters the output as we want. For example, we want to get the documents about the IT products only. 

<i>Note that we need to add quotation marks to quote the keys mentioned in the argument. i.e., `'type'` in the following example.</i>

In [ ]:
data = list(products.find({'type': 'it'}))

pprint(data)
    

## Insertion, Update, and Deletion

Same as `mongosh`, `PyMongo` provides functions for manipulating the documents including insertion, update, and deletion.

To insert a new document, we use the `insert_one` function.

In [ ]:
products.insert_one({'_id': 12, 'name': 'mouse', 'qty': 20, 'type': 'it'})

Let's check whether it works or not!

In [ ]:
data = products.find_one({'_id': 12})
pprint(data)

We can also insert multiple documents at once using the `insert_many` function.

In [ ]:
products.insert_many([
    {'_id': 200, 'name': 'something 1', 'qty': 500, 'type': 'unknown'},
    {'_id': 201, 'name': 'something 2', 'qty': 500, 'type': 'unknown'},
    {'_id': 202, 'name': 'something 3', 'qty': 500, 'type': 'unknown'},
    {'_id': 203, 'name': 'something 4', 'qty': 500, 'type': 'unknown'},
    {'_id': 204, 'name': 'something 5', 'qty': 500, 'type': 'unknown'}
])

data = list(products.find())
pprint(data)

We even can get a value of the specific field in a document. For example, we want to get **the quantity of the product with ID 204**.

In [ ]:
data = products.find_one({'_id': 204})
qty = data['qty']
pprint(qty)

Let's try to update the quantity of product 204 with `qty + 1000`.

<i>Note that the `update_one` function will update the first match only.</i>

In [ ]:
products.update_one({'_id': 204}, {'$set': {'qty': qty + 1000}})
data = products.find_one({'_id': 204})
pprint(data)

With the `update_many` function, we can update multiple documents at once. For example, we change type "unknown" to "UNKNOWN".

In [ ]:
products.update_many({'type': 'unknown'}, {'$set':{'type':'UNKNOWN'}})
data = list(products.find())
pprint(data)

To delete documents, we use the `delete_one` or `delete_many` functions.

<i>Same situation as `update_one`, the `delete_one` function takes effect on the first match only.</i>

In [ ]:
products.delete_one({'type': 'UNKNOWN'})
data = list(products.find())
pprint(data)

To delete all products with "UNKNOWN" type, we should use the `delete_many` function instead.

In [ ]:
products.delete_many({'type':'UNKNOWN'})
data = list(products.find())
pprint(data)

## Importing Data

In the previous section, we inserted documents manually. Of course, we may store a lot of data in data files and want to import them into MongoDB. We are going to learn how to import the data from a text file using the Python code.

Let's download the data file named `annual-enterprise-survey-2021-financial-year-provisional-csv.txt` and store it in the comp7095 folder on the desktop.

The data file is originally from Stats NZ Tatauranga Aotearoa. 

https://stats.govt.nz/assets/Uploads/Annual-enterprise-survey/Annual-enterprise-survey-2021-financial-year-provisional/Download-data/annual-enterprise-survey-2021-financial-year-provisional-csv.csv

To simplify the Python code for loading the data from the data file, the content of the data file is pre-processed using Microsoft Excel. The changes include:
1. Removed the comma from column `Value`.
2. Saved the file in Tab Delimited Text format (`.txt`).

First, we use the `open` function to open the data file in `read-only` mode.

In [ ]:
file = open('annual-enterprise-survey-2021-financial-year-provisional-csv.txt', 'r')

#### Process the Heading Line
The first line in the data file is a heading line. We use the `readline` function to read it and name it `headings`.

In [ ]:
headings = file.readline()

Let's see what we get from the file! The line contains the headings separated by a tab (`\t`) and ends with a newline character (`\n`).

In [ ]:
headings

We delete the `\n` and split the line using the `\t`. So, we have a list that contains the headings. Then, `headings` now refers to a list of the headings.

In [ ]:
headings = headings.replace('\n', '').split('\t')
headings

#### Process the Data Lines
Next, we do the same thing for the data part. The procedure is that:
1. create an empty list and a counter.

In [ ]:
temp = []
count = 0

2. Read all remaining lines from the file and name it `lines`. After that we close the file.

In [ ]:
lines = file.readlines()
file.close()

3. Repeatedly do the following for each line:
 1. Get a line from the list, delete the `\n` and split it using `\t`. 
 1. Create a new dictionary. 
 1. Increate the count by 1 and store it in the dictionary as an ID of the document we are going to create. 
 1. Use a for loop to put the values with corresponding headings to the dictionary. 
 1. Put the dictionary to the list. 

In [ ]:
for line in lines:
    values = line.replace('\n', '').split('\t')
    doc = {}
    count += 1
    doc['_id'] = count
    for i in range(len(headings)):
        doc[headings[i]] = values[i]
    temp.append(doc)

4. We get a collection named `companies`. To guarantee it is empty, we call its `delete_many` function to delete everything.

In [ ]:
companies = db['companies']
companies.delete_many({})

5. Call the `insert_many` function with the list to create documents in MongoDB.

In [ ]:
companies.insert_many(temp)

6. Verify the insertion by getting the first 5 documents using the `find` function and `limit` function.

In [ ]:
data = list(companies.find().limit(5))
pprint(data)

7. By default, the data type of each field is string. We need to convert the `Value` field to integer. We use a for loop to retrieve every document and use the `update_one` function to update the value of the `Value` field.

In [ ]:
for d in data:
    id = d['_id']
    try:
        value = int(d['Value'])
    except:
        value = 0      ## We set the value to zero if the value cannot be converted to an integer.
        
    companies.update_one({'_id':id}, {'$set':{'Value':value}})
    

Done! Let's see what we have!

In [ ]:
data = list(companies.find().limit(5))
pprint(data)

We can use the comparison keywords to filter the documents, just like what we learnt about `mongosh`.

In [ ]:
data = list(companies.find({'Value':{'$gt': 100000}}).limit(5))
pprint(data)

We can also sort the output by using the `sort` function.

In [ ]:
data = list(companies.find({'Value':{'$lte': -1000}}).sort('Value', -1).limit(5))
pprint(data)

## Aggregate

In Python, PyMongo package provides the `aggregate` function, similar to the `aggregate` function of `mongosh`. But, the argument of the `aggregate` function must be a list of dictionaries.

Let's see the following example. We use the `aggregate` function to get the count of the documents in the `companies` collection:

In [ ]:
data = list(companies.aggregate([{'$group':{'_id':'', 'count(*)':{'$sum':1}}}]))
pprint(data)

You can think the code above as `SELECT count(*) FROM companies`.

The following example shows how to retrieve the document counts of each year. You can think it as `SELECT Years, count(*) FROM companies GROUP BY Years`.

In [ ]:
data = list(companies.aggregate([{'$group':{'_id':'$Year', 'count':{'$sum':1}}}]))
pprint(data)

In the following example, we add one more dictionary to sort the output by `_id` in descending order.

In [ ]:
data = list(companies.aggregate([{'$group':{'_id':'$Year', 'count':{'$sum':1}}}, {'$sort': {'_id': -1}}]))
pprint(data)

You can also try other aggregates:


|Aggregate||Description|
|---|---|---|
|\$sum||Calculate the total|
|\$avg||Find the average|
|\$min||Find the minimum value|
|\$max||Find the maximum value|
|\$push||Add the value to a list, duplicates are allowed|
|\$addToSet||Add the value to a set, duplicates will not be added|
|\$first||Find the first match only|
|\$last||Find the last match only|


## Exporting to JSON file

Since the documents are represented as a list of the dictionaries in Python, we can convert them to JSON format and save the result to a data file. The following is the procedure to do so.

First, we retrieve all documents from the collection.

In [ ]:
data = list(companies.find())

We use the `dumps` function provided by JSON package to convert the list of the documents to a JSON-String.

In [ ]:
import json
json_str = json.dumps(data)

Then, we open a file with the `write` mode and write the string to the file. 

In [ ]:
fout = open('companies.json', 'w')
fout.write(json_str)
fout.close()

## Importing JSON file

In the previous part, we learnt how to load the data from a text file in Tab Delimited Text format. Now, we are going to learn how to load the data from a JSON file.

Before loading the data into MongoDB, we ensure that collection `companies` is empty. Therefore, we can the `delete_many` function to delete all documents.

In [ ]:
companies.delete_many({})
data = list(companies.find())
print(data)

First, we open the data file from using the open function with the `read-only` mode ('r'), and load the line from the data file. Assume that the content contains a single line only. Hint, you may use readline() to read this line.

### Task 1: Open ("companies.json") file in correct mode

We then use the loads function provided by JSON package to convert the data from JSON-string format to a list.

In [ ]:
import json
data = json.loads(json_str)

Now, we can call the insert_many function to insert documents.

### Task 2: insert your data to companies 

Let's check whether the collection contains the new documents or not.

### Task 3: use find() method to check the contained data
please display the first five results in your results